<a href="https://colab.research.google.com/github/AlcidesChewe/TF_MENSAJES_NEGATIVOS/blob/main/Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd # read the csv
import re # regex to detect username, url, html entity
import nltk # to use word tokenize (split the sentence into words)
from nltk.corpus import stopwords # to remove the stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

from keras.utils import to_categorical
from keras import backend as K
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import numpy as np

In [19]:
# initiate the path and read it
dataset_path = "./labeled_data.csv"
df = pd.read_csv(dataset_path)
df.head()

# dataset shape to know how many tweets in the datasets
print(f"num of tweets: {df.shape}")

# extract the text and labels
tweet = list(df['tweet'])
labels = list(df['class'])

num of tweets: (24783, 7)


In [20]:
stop_words = set(stopwords.words('english'))
stop_words.add("rt")

# remove html entity:
def remove_entity(raw_text):
    entity_regex = r"&[^\s;]+;"
    text = re.sub(entity_regex, "", raw_text)
    return text

# change the user tags
def change_user(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "user", raw_text)

    return text

# remove urls
def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)

    return text

# remove unnecessary symbols
def remove_noise_symbols(raw_text):
    text = raw_text.replace('"', '')
    text = text.replace("'", '')
    text = text.replace("!", '')
    text = text.replace("`", '')
    text = text.replace("..", '')

    return text

# remove stopwords
def remove_stopwords(raw_text):
    tokenize = nltk.word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = " ".join(text)

    return text

## this function in to clean all the dataset by utilizing all the function above
def preprocess(datas):
    clean = []
    # change the @xxx into "user"
    clean = [change_user(text) for text in datas]
    # remove emojis (specifically unicode emojis)
    clean = [remove_entity(text) for text in clean]
    # remove urls
    clean = [remove_url(text) for text in clean]
    # remove trailing stuff
    clean = [remove_noise_symbols(text) for text in clean]
    # remove stopwords
    clean = [remove_stopwords(text) for text in clean]

    return clean

In [21]:
clean_tweet = preprocess(tweet)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(clean_tweet, labels, test_size=0.2, random_state=42)

In [23]:
## Tokenizing -> basically we use tokenisation for many things, its commonly used for feature extraction in preprocessing. btw idk how it works as feature extraction tho :(
# declare the tokenizer
tokenizer = Tokenizer()
# build the vocabulary based on train dataset
tokenizer.fit_on_texts(X_train)
# tokenize the train and test dataset
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# vocabulary size (num of unique words) -> will be used in embedding layer
vocab_size = len(tokenizer.word_index) + 1

In [24]:
# Cargar GloVe embeddings
embedding_index = dict()
with open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Crear la matriz de embedding
embedding_matrix = np.zeros((vocab_size, 100))  # Asegúrate de que la dimensión aquí corresponda con la de GloVe
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        # Las palabras no encontradas en GloVe serán vectores de ceros.
        embedding_matrix[i] = embedding_vector


In [25]:
## Padding -> to uniform the datas
max_length = max(len(seq) for seq in X_train)

# to test an outlier case (if one of the test dataset has longer length)
for x in X_test:
    if len(x) > max_length:
        print(f"an outlier detected: {x}")

X_train = pad_sequences(X_train, maxlen = max_length)
X_test = pad_sequences(X_test, maxlen = max_length)

In [26]:
# create hot_labels (idk whty tapi ini penting, kalo ga bakal error)
y_test = to_categorical(y_test, num_classes=3)
y_train = to_categorical(y_train, num_classes=3)

In [27]:
# another look on the number of tweet in test and training data

print(f"num test tweet: {y_test.shape[0]}")
print(f"num train tweet: {y_train.shape[0]}")

num test tweet: 4957
num train tweet: 19826


In [28]:
# evaluation metrics
# credits/source:
"""
https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
"""
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [29]:
# change dis if u want
#output_dim = 200

# LSTM model architechture (CNN + LSTM)
#model = Sequential([
    # embedding layer is like idk
  #  Embedding(vocab_size, output_dim, input_length=max_length),
    # lstm for xxx
  #  LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    # dropout to prevent overfitting
 #   Dropout(0.5),
    # dense to connect the previous output with current layer
 #   Dense(128, activation="relu"),
    # dropout to prevent overfitting
 #   Dropout(0.5),
    # this is output layer, with 3 class (0, 1, 2)
 #   Dense(3, activation="softmax"),
#])

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1,precision, recall])

# LSTM model architecture (CNN + LSTM)
model = Sequential([
    # Cambia la capa de embedding para usar los embeddings preentrenados de GloVe
    Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False),
    # lstm for xxx
    LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    # dropout to prevent overfitting
    Dropout(0.5),
    # dense to connect the previous output with current layer
    Dense(128, activation="relu"),
    # dropout to prevent overfitting
    Dropout(0.5),
    # this is output layer, with 3 class (0, 1, 2)
    Dense(3, activation="softmax"),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1, precision, recall])



In [30]:
# checking the model parameters
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 26, 100)           1867100   
                                                                 
 lstm_1 (LSTM)               (None, 64)                42240     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1918047 (7.32 MB)
Trainable params: 5094

In [31]:
# Train the model
model_history = model.fit(
    X_train,
    y_train,
    batch_size = 64,
    epochs=10,
    validation_data=(X_test, y_test)
)

Epoch 1/10
310/310 [==============================] - 27s 70ms/step - loss: 0.5093 - accuracy: 0.8220 - f1: 0.8087 - precision: 0.8405 - recall: 0.7858 - val_loss: 0.3833 - val_accuracy: 0.8594 - val_f1: 0.8598 - val_precision: 0.8721 - val_recall: 0.8479
Epoch 2/10
310/310 [==============================] - 24s 77ms/step - loss: 0.3867 - accuracy: 0.8615 - f1: 0.8600 - precision: 0.8783 - recall: 0.8428 - val_loss: 0.3232 - val_accuracy: 0.8763 - val_f1: 0.8768 - val_precision: 0.8950 - val_recall: 0.8596
Epoch 3/10
310/310 [==============================] - 21s 68ms/step - loss: 0.3522 - accuracy: 0.8713 - f1: 0.8687 - precision: 0.8863 - recall: 0.8521 - val_loss: 0.3130 - val_accuracy: 0.8780 - val_f1: 0.8756 - val_precision: 0.8978 - val_recall: 0.8550
Epoch 4/10
310/310 [==============================] - 23s 75ms/step - loss: 0.3332 - accuracy: 0.8764 - f1: 0.8753 - precision: 0.8920 - recall: 0.8595 - val_loss: 0.3054 - val_accuracy: 0.8836 - val_f1: 0.8841 - val_precision: 0.90

In [ ]:
model.save("./pretrained")